In [ ]:
import os
from PIL import Image, ExifTags
import csv
import argparse


def extract_exif(file_path):
    """
    Extract EXIF metadata from an image file.
    :param file_path: Path to the image file
    :return: Dictionary containing EXIF metadata
    """
    try:
        image = Image.open(file_path)
        exif_data = {
            ExifTags.TAGS.get(tag, tag): value
            for tag, value in image._getexif().items()
            if tag in ExifTags.TAGS
        }
        return exif_data
    except Exception as e:
        print(f"Error reading EXIF data from {file_path}: {e}")
        return {}


def generate_report(metadata_list, output_file):
    """
    Generate a CSV report from the extracted metadata.
    :param metadata_list: List of dictionaries containing metadata
    :param output_file: Path to the output CSV file
    """
    try:
        with open(output_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=metadata_list[0].keys())
            writer.writeheader()
            writer.writerows(metadata_list)
        print(f"Metadata report saved to {output_file}")
    except Exception as e:
        print(f"Error writing report to {output_file}: {e}")


def main(directory, output_file=None):
    """
    Main function to scan a directory for images and extract their EXIF metadata.
    :param directory: Directory containing image files
    :param output_file: Optional path to save the metadata as a CSV file
    """
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return

    metadata_list = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith(('.jpg', '.jpeg', '.tiff')):
                print(f"Processing {file_path}")
                exif_data = extract_exif(file_path)
                if exif_data:
                    metadata_list.append({"File": file, **exif_data})

    if metadata_list:
        if output_file:
            generate_report(metadata_list, output_file)
        else:
            for metadata in metadata_list:
                print(metadata)
    else:
        print("No image files with EXIF metadata found.")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Extract EXIF metadata from image files in a directory.")
    parser.add_argument("directory", help="Path to the directory containing image files.")
    parser.add_argument("--output", help="Path to save the metadata report as a CSV file.")
    args = parser.parse_args()

    main(args.directory, args.output)
